In [ ]:
## simple workig but not logic 
import pandas as pd
import re

# Load the previously extracted data from Excel
df_table = pd.read_excel("extracted_table.xlsx", header=None)

# Skip the first two rows and extract the remaining data
df_table_remaining = df_table.iloc[2:]

# Convert the DataFrame to a single string to process all rows together
all_text = ' '.join(df_table_remaining[0].dropna().astype(str).tolist())

# Split the text by "O " followed by a space, but keep the delimiter in the result
rows = re.split(r'(O )', all_text)
rows = [''.join(pair) for pair in zip(rows[1::2], rows[2::2])]

# Function to clean and split lines
def clean_and_split(line):
    if isinstance(line, str):
        # Logic 1: Handle alphanumeric data followed by text and $ sign
        pattern1 = re.compile(r'(\S+)\s+(.*?)\s+\$(\d+\.?\d*)')
        match1 = pattern1.search(line)
        if match1:
            alphanumeric = match1.group(1)
            text_between = match1.group(2).strip().replace(' ', '_')
            amount = f"${match1.group(3)}"
            
            # Split the remaining part of the line
            remaining_part = line[match1.end():].strip()
            additional_columns = remaining_part.split() if remaining_part else []
            
            # Construct the final output with additional columns
            result = [alphanumeric, text_between, amount] + additional_columns
            return ";".join(result)

        # Logic 2: Handle 'provider' followed by a 2-digit number and $ sign
        pattern2 = re.compile(r'(provider)\s*(\d{2})?\s+\$(\d+\.?\d*)', re.IGNORECASE)
        match2 = pattern2.search(line)
        if match2:
            provider = match2.group(1).capitalize()
            digits = match2.group(2) if match2.group(2) else '10'  # Insert '10' if 2-digit number is not found
            amount = f"${match2.group(3)}"
            
            # Split the remaining part of the line
            remaining_part = line[match2.end():].strip()
            additional_columns = remaining_part.split() if remaining_part else []
            
            # Construct the final output with additional columns
            result = [provider, digits, amount] + additional_columns
            return ";".join(result)

    return line

# Function to merge lines if a line might be a continuation of the previous line
def merge_lines(data):
    merged_data = []
    for i, line in enumerate(data):
        if i > 0 and isinstance(line, str) and line.islower():
            # Merge with the previous line
            merged_data[-1] += " " + line.strip()
        else:
            merged_data.append(line)
    return merged_data

# Apply the merge_lines function to combine split lines
merged_table = merge_lines(rows)

# Apply the cleaning function to each row in the merged data
df_table_cleaned = pd.Series(merged_table).apply(clean_and_split)

# Split the semicolon-separated values into separate columns
df_table_split = df_table_cleaned.str.split(';', expand=True)

# Handle error cases where the split might not produce enough columns
df_table_split = df_table_split.fillna('')

# Save the cleaned and split data to a new Excel file
df_table_split.to_excel("cleaned_extracted_table.xlsx", index=False, header=False)

print("Data cleaning and splitting complete. Saved to cleaned_extracted_table.xlsx")


In [ ]:
import pandas as pd
import re

# Load the data from the Excel file
df_table = pd.read_excel("extracted_table.xlsx", header=None)

# Skip the first two rows and extract the remaining data
df_table_remaining = df_table.iloc[2:]

# Convert the DataFrame to a single string to process all rows together
all_text = ' '.join(df_table_remaining[0].dropna().astype(str).tolist())

# Split the text by "O " followed by a space, but keep the delimiter in the result
rows = re.split(r'(O )', all_text)
rows = [''.join(pair) for pair in zip(rows[1::2], rows[2::2])]

# Convert the split rows into a DataFrame
df_rows = pd.DataFrame(rows, columns=['RawText'])

# Function to clean and split lines based on the provided patterns
def clean_and_split(line):
    if isinstance(line, str):
        # Logic 1: Handle alphanumeric data followed by text and $ sign
        pattern1 = re.compile(r'(\S+)\s+(.*?)\s+\$(\d+\.?\d*)')
        match1 = pattern1.search(line)
        if match1:
            alphanumeric = match1.group(1)
            text_between = match1.group(2).strip().replace(' ', '_')
            amount = f"${match1.group(3)}"
            
            # Split the remaining part of the line
            remaining_part = line[match1.end():].strip()
            additional_columns = remaining_part.split() if remaining_part else []
            
            # Construct the final output with additional columns
            result = [alphanumeric, text_between, amount] + additional_columns
            return ";".join(result)

        # Logic 2: Handle 'provider' followed by a 2-digit number and $ sign
        pattern2 = re.compile(r'(provider)\s*(\d{2})?\s*(.*?)?\s+\$(\d+\.?\d*)', re.IGNORECASE)
        match2 = pattern2.search(line)
        if match2:
            provider = match2.group(1).capitalize()
            digits = match2.group(2) if match2.group(2) else '10'  # Insert '10' if 2-digit number is not found
            intermediate_text = match2.group(3).strip() if match2.group(3) else ''
            amount = f"${match2.group(4)}"
            
            # Split the remaining part of the line
            remaining_part = line[match2.end():].strip()
            additional_columns = remaining_part.split() if remaining_part else []
            
            # Construct the final output with additional columns
            result = [provider, digits, intermediate_text, amount] + additional_columns
            return ";".join(filter(None, result))  # filter(None, result) removes empty strings
        
    return line

# Apply the cleaning function to each row in the DataFrame
df_rows['CleanedText'] = df_rows['RawText'].apply(clean_and_split)

# Split the cleaned text into separate columns
df_cleaned_split = df_rows['CleanedText'].str.split(';', expand=True)

# Handle error cases where the split might not produce enough columns
df_cleaned_split = df_cleaned_split.fillna('')

# Save the cleaned and split data to a new Excel file
df_cleaned_split.to_excel("cleaned_extracted_table.xlsx", index=False, header=False)

print("Data cleaning and splitting complete. Saved to cleaned_extracted_table.xlsx")


In [ ]:
import pandas as pd
import re

# Load the data from the Excel file
df_table = pd.read_excel("extracted_table.xlsx", header=None)

# Skip the first two rows and extract the remaining data
df_table_remaining = df_table.iloc[2:].dropna()

# Convert the DataFrame to a single string to process all rows together
all_text = ' '.join(df_table_remaining[0].astype(str).tolist())

# Split the text by "O " followed by a space, but keep the delimiter in the result
rows = re.split(r'(O\s)', all_text)
rows = [''.join(pair) for pair in zip(rows[1::2], rows[2::2])]

# Convert the split rows into a DataFrame
df_rows = pd.DataFrame(rows, columns=['RawText'])

# Function to clean and split lines based on the provided patterns
def clean_and_split(line):
    if isinstance(line, str):
        # Logic 1: Handle alphanumeric data followed by text and $ sign
        pattern1 = re.compile(r'(\S+)\s+(.*?)\s+\$(\d+\.?\d*)')
        match1 = pattern1.search(line)
        if match1:
            alphanumeric = match1.group(1)
            description = match1.group(2).strip().replace(' ', '_')
            amount = f"${match1.group(3)}"
            
            # Split the remaining part of the line
            remaining_part = line[match1.end():].strip()
            additional_columns = remaining_part.split() if remaining_part else []
            
            # Construct the final output with additional columns
            result = [alphanumeric, description, amount] + additional_columns
            return ";".join(result)
        
        # Logic 2: Handle 'provider' followed by a 2-digit number and $ sign
        pattern2 = re.compile(r'(provider)\s*(\d{2})?\s*(.*?)?\s+\$(\d+\.?\d*)', re.IGNORECASE)
        match2 = pattern2.search(line)
        if match2:
            provider = match2.group(1).capitalize()
            digits = match2.group(2) if match2.group(2) else '10'  # Insert '10' if 2-digit number is not found
            intermediate_text = match2.group(3).strip().replace(' ', '_') if match2.group(3) else ''
            amount = f"${match2.group(4)}"
            
            # Split the remaining part of the line
            remaining_part = line[match2.end():].strip()
            additional_columns = remaining_part.split() if remaining_part else []
            
            # Construct the final output with additional columns
            result = [provider, digits, amount] + additional_columns
            return ";".join(result)

    # Handle cases with no matches or different format
    return line

# Apply the cleaning function to each row in the DataFrame
df_rows['CleanedText'] = df_rows['RawText'].apply(clean_and_split)

# Split the cleaned text into separate columns
df_cleaned_split = df_rows['CleanedText'].str.split(';', expand=True)

# Handle error cases where the split might not produce enough columns
df_cleaned_split = df_cleaned_split.fillna('')

# Save the cleaned and split data to a new Excel file
df_cleaned_split.to_excel("cleaned_extracted_table.xlsx", index=False, header=False)

print("Data cleaning and splitting complete. Saved to cleaned_extracted_table.xlsx")
###not workingg

In [ ]:
import pandas as pd
import re

# Load the data from the Excel file
df_table = pd.read_excel("extracted_table.xlsx", header=None)

# Skip the first two rows and extract the remaining data
df_table_remaining = df_table.iloc[2:]

# Convert the DataFrame to a single string to process all rows together
all_text = ' '.join(df_table_remaining[0].dropna().astype(str).tolist())

# Split the text by "O " followed by a space, but keep the delimiter in the result
rows = re.split(r'(O\s)', all_text)
rows = [''.join(pair) for pair in zip(rows[1::2], rows[2::2])]

# Convert the split rows into a DataFrame
df_rows = pd.DataFrame(rows, columns=['RawText'])

# Function to handle alphanumeric data and description
def handle_alphanumeric(line):
    if isinstance(line, str):
        # Handle alphanumeric data followed by text and $ sign
        pattern = re.compile(r'(\S+)\s+(.*?)\s+\$(\d+\.?\d*)')
        match = pattern.search(line)
        if match:
            alphanumeric = match.group(1)
            description = match.group(2).strip().replace(' ', '_')
            amount = f"${match.group(3)}"
            
            # Split the remaining part of the line
            remaining_part = line[match.end():].strip()
            additional_columns = remaining_part.split() if remaining_part else []
            
            # Construct the final output with additional columns
            result = [alphanumeric, description, amount] + additional_columns
            return ";".join(result)
        
    return line

# Function to handle provider data
def handle_provider(line):
    if isinstance(line, str):
        # Handle 'provider' followed by a 2-digit number and $ sign
        pattern = re.compile(r'(provider)\s*(\d{2})?\s*(.*?)?\s+\$(\d+\.?\d*)', re.IGNORECASE)
        match = pattern.search(line)
        if match:
            provider = match.group(1).capitalize()
            digits = match.group(2) if match.group(2) else '10'  # Insert '10' if 2-digit number is not found
            intermediate_text = match.group(3).strip().replace(' ', '_') if match.group(3) else ''
            amount = f"${match.group(4)}"
            
            # Split the remaining part of the line
            remaining_part = line[match.end():].strip()
            additional_columns = remaining_part.split() if remaining_part else []
            
            # Construct the final output with additional columns
            result = [provider, digits, amount] + additional_columns
            return ";".join(result)

    return line

# Apply the cleaning functions to each row in the DataFrame
df_rows['CleanedText'] = df_rows['RawText'].apply(lambda x: handle_alphanumeric(x) if 'provider' not in x.lower() else handle_provider(x))

# Split the cleaned text into separate columns
df_cleaned_split = df_rows['CleanedText'].str.split(';', expand=True)

# Handle error cases where the split might not produce enough columns
df_cleaned_split = df_cleaned_split.fillna('')

# Save the cleaned and split data to a new Excel file
df_cleaned_split.to_excel("cleaned_extracted_table.xlsx", index=False, header=False)

print("Data cleaning and splitting complete. Saved to cleaned_extracted_table.xlsx")
